In [1]:
import requests
import pandas as pd
import os  
import time
import urllib.request
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
from urllib.parse import urlparse

In [2]:
term = "bouguereau"

In [3]:
#Make request
api_url = "https://collectionapi.metmuseum.org/public/collection/v1"
endpoint = "/search"
url = api_url+endpoint
#Dictionary of parameters – check the documentation for parameters to your liking
params = {'q': term, 
          'hasImages': True,
          'departmentID': 11, #in the documentation I found that the ID for European Paintings is 11
          # 'dateBegin' : 1800,
          # 'dateEnd': 1905
          }
response = requests.get(url, params = params)
print(response.url)
print("STATUS:", response.status_code)

https://collectionapi.metmuseum.org/public/collection/v1/search?q=bouguereau&hasImages=True&departmentID=11
STATUS: 200


In [4]:
temp_json = response.json()
temp_json

{'total': 25,
 'objectIDs': [192770,
  435848,
  453385,
  451725,
  452102,
  329077,
  39901,
  485416,
  204587,
  436098,
  439327,
  437202,
  437173,
  452032,
  437216,
  435702,
  435621,
  438821,
  436105,
  464118,
  451287,
  488221,
  450761,
  39895,
  552008]}

In [5]:
results = []

api_url = "https://collectionapi.metmuseum.org/public/collection/v1/objects/"

for i in temp_json['objectIDs']:
    url = api_url + str(i)
    response = requests.get(url).json()
    results.append(response)

print(len(results))

25


In [6]:
#using pandas, we can use json_normalize to put our data into a dataframe!
df = pd.json_normalize(results)

df = df[['objectID', 'primaryImage','department','objectName','title','artistDisplayName','objectDate']]

# df = df[df['primaryImage'].notna()]
df

,objectID,primaryImage,department,objectName,title,artistDisplayName,objectDate
0,192770,https://images.metmuseum.org/CRDImages/es/orig...,European Sculpture and Decorative Arts,Relief,Saint Bridget of Sweden Receiving the Rule of ...,Agostino d'Antonio di Duccio,1459
1,435848,https://images.metmuseum.org/CRDImages/ep/orig...,European Paintings,Painting,The Birth of the Virgin,Fra Carnevale (Bartolomeo di Giovanni Corradini),1467
2,453385,https://images.metmuseum.org/CRDImages/is/orig...,Islamic Art,Non-illustrated manuscript,Anthology of Persian Poetry in Oblong Format (...,Sultan Muhammad Nur,dated 905 AH/1499–1500 CE
3,451725,https://images.metmuseum.org/CRDImages/is/orig...,Islamic Art,Folio from an illustrated manuscript,"""The Concourse of the Birds"", Folio 11r from a...",Habiballah of Sava,ca. 1600
4,452102,https://images.metmuseum.org/CRDImages/is/orig...,Islamic Art,Period room,Damascus Room,,dated 1119 AH/1707 CE
5,329077,https://images.metmuseum.org/CRDImages/an/orig...,Ancient Near Eastern Art,Sculpture,Head of a ruler,,ca. 2300–2000 BCE
6,39901,https://images.metmuseum.org/CRDImages/as/orig...,Asian Art,Handscroll,Night-Shining White,Han Gan,ca. 750
7,485416,,Modern and Contemporary Art,Painting,Street Story Quilt,Faith Ringgold,1985
8,204587,https://images.metmuseum.org/CRDImages/es/orig...,European Sculpture and Decorative Arts,Bust,Emperor Antoninus Pius,Antico (Pier Jacopo Alari Bonacolsi),1519–24
9,436098,https://images.metmuseum.org/CRDImages/ep/orig...,European Paintings,Painting,The Crucifixion,Gerard David,ca. 1495


In [7]:

def merge(list1, list2):    
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

def download_image(img_list, file_path):
    # print(image_list[1])
    # img_list[1].decode("utf-8")
    filename = '{}.jpg'.format(img_list[0])
    fullpath = '{}{}'.format(file_path, filename)
    try: 
        urllib.request.urlretrieve(img_list[1], fullpath) 
    except URLError:
        print(f"Error downloading: {img_list[0]}")
        errorlist.append(img_list[0])
    except HTTPError:
        print(f"Error downloading: {img_list[0]}")
        errorlist.append(img_list[0])

In [8]:
#this returns a list of tuples of object ID and their image URls
image_url = df['primaryImage'].to_list()

# image_url = image_url.replace("\\", "/")
image_url = [str(str(i).replace("\\", "/").replace(" ", "")) for i in image_url]

image_id = df['objectID'].to_list()
image_list = merge(image_id, image_url)

In [9]:
print(image_url[6])

https://images.metmuseum.org/CRDImages/as/original/DP153705.jpg


In [10]:
directory = term
file_path = os.path.join("data/", directory + "/")  
os.mkdir(file_path)  

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data/bouguereau/'

In [11]:


#download images, BE CAREFUL about how many images you have! i'd suggest to stop at 100. simply index image_list like so: image_list[:n]

#the errorlist flags what images did not download for whatever reason
errorlist= []

t1 = time.perf_counter()

for url in image_list:
    # url.decode('utf-8')
    try:
         download_image(url, file_path)
    except UnicodeEncodeError:
        print("probably has a - somewhere")
    except ValueError:
        print("no link for some reason")
    
        
t2 = time.perf_counter()

print(f'Finished in {t2-t1} seconds')

(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
no link for some reason
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP109484.jpg')
(435848, 'https://images.metmuseum.org/CRDImages/ep/original/DP1